Document Classification

1. Text pre-processing and exploration:

In [45]:
import os
import nltk
import sklearn
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from os import listdir
from os.path import isfile, join    
import sklearn.datasets as sk


In [47]:
# here you should put the directory path
path='C:\\Users\\Nogah\\Documents\\אוניברסיטה\\שנה ד\\סמסטר א\\היבטים מעשיים לכריית תוכן באינטרנט\\ohsumed-first-20000-docs'

dirpath=path
trainDirs=[]
testDirs=[]
trainFiles=[]
testFiles=[]
# for table
Category=[]
FileName=[]
Path=[]
Text=[]
classNumOfDocs=[]
trainInfo=[] #[fileName,cleanText,class
testInfo=[]

stopWords=(stopwords.words('english'))
stopWords += {',','.',':',';','(',')','{','}','[',']'}#set(string.punctuation)

In [48]:
# get train and test dirs
trainDirs=os.listdir(dirpath+'\\training')
testDirs=os.listdir(dirpath+'\\test')

In [49]:
# pre-process
def cleanText(text):
    # tokenize
    tokenized = word_tokenize(text)
    # remove stop words
    filtered_sentence = [w for w in tokenized if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    # stemming
    ps = PorterStemmer()
    index=0
    for i in range(len(filtered_sentence)-1):
        if len(text) > 1:
            try:
                filtered_sentence[index] = ps.stem(filtered_sentence[i])
            except Exception as inst:
                filtered_sentence[index]=filtered_sentence[i]
    return " ".join(filtered_sentence)


In [50]:
# get train and test dirs
for dir in trainDirs:
    for file in os.listdir(dirpath+'\\training\\'+dir):
        trainFiles.append(file)
        Category.append(dir)
        FileName.append(file)
        Path.append(dirpath+'\\training\\'+dir+'\\'+file)
        # Open a file: fileReader
        fileReader = open(dirpath+'\\training\\'+dir+'\\'+file,mode='r')
        # read all lines at once
        text = fileReader.read().lower()
        # close the file
        fileReader.close()

        cleanedText=cleanText(text)
        trainInfo.append([file,cleanedText,dir,'training'])

for dir in testDirs:
    for file in os.listdir(dirpath+'\\test\\'+dir):
        testFiles.append(file)
        Category.append(dir)
        FileName.append(file)
        Path.append(dirpath+'\\test\\'+dir+'\\'+file)
        # Open a file: fileReader
        fileReader = open(dirpath+'\\test\\'+dir+'\\'+file,mode='r')
        # read all lines at once
        text = fileReader.read().lower()
        # close the file
        fileReader.close()

        cleanedText = cleanText(text)
        testInfo.append([file, cleanedText, dir, 'test'])

In [51]:
#get number of classes and number of docs for each class
docInfoDF=pd.DataFrame(trainInfo, columns=['fileName','cleanText','class','group'])
allClasses=np.unique(docInfoDF['class'])
print ('# of categories: ',allClasses.size)
numOfFiles=[]
words_per_class = []
termsFreqPerClass=[]
for currClass in allClasses:
    temp=(docInfoDF['class']).value_counts()[currClass]
    numOfFiles.append([currClass, temp])
    #get terms distibution
    currFiles=docInfoDF.loc[docInfoDF['class']==currClass]
    termsFreq=pd.Series(" ".join(currFiles['cleanText']).split()).value_counts()[:10]
    # termsFreqPerClass.append([currClass,zip(termsFreq.index,termsFreq)])
    zipedFreq=zip(termsFreq.index, termsFreq)
    zippedList=list(zipedFreq)
    termsFreqPerClass.append([currClass]+zippedList)

# print number of files prt category
numOfFilesDF=pd.DataFrame(numOfFiles,columns=['Class','#OfFiles'])
print(numOfFilesDF)
# print terms frequency
pd.DataFrame(termsFreqPerClass, columns=['Class','term 1','term 2','term 3','term 4','term 5','term 6','term 7','term 8','term 9','term 10'])

# of categories:  23
   Class  #OfFiles
0    C01       423
1    C02       158
2    C03        65
3    C04      1163
4    C05       283
5    C06       588
6    C07       100
7    C08       473
8    C09       125
9    C10       621
10   C11       162
11   C12       491
12   C13       281
13   C14      1249
14   C15       215
15   C16       200
16   C17       295
17   C18       388
18   C19       191
19   C20       525
20   C21       546
21   C22        92
22   C23      1799


,Class,term 1,term 2,term 3,term 4,term 5,term 6,term 7,term 8,term 9,term 10
0,C01,"(patients, 832)","(infection, 408)","(treatment, 258)","(disease, 196)","(group, 194)","(therapy, 171)","(may, 167)","(less, 160)","(clinical, 156)","(cases, 154)"
1,C02,"(patients, 248)","(infection, 186)","(virus, 180)","(human, 132)","(immunodeficiency, 101)","(hiv, 89)","(disease, 82)","(cells, 70)","(clinical, 65)","(aids, 60)"
2,C03,"(patients, 91)","(infection, 52)","(disease, 43)","(treatment, 33)","(cases, 28)","(falciparum, 27)","(malaria, 26)","(levels, 23)","(therapy, 22)","(patient, 21)"
3,C04,"(patients, 2546)","(tumor, 891)","(cancer, 890)","(cells, 862)","(cell, 769)","(carcinoma, 705)","(tumors, 573)","(treatment, 554)","(disease, 503)","(cases, 491)"
4,C05,"(patients, 509)","(bone, 208)","(may, 107)","(treatment, 107)","(years, 105)","(results, 102)","(syndrome, 94)","(arthritis, 94)","(disease, 93)","(study, 92)"
5,C06,"(patients, 1682)","(disease, 489)","(less, 313)","(treatment, 274)","(gastric, 264)","(may, 243)","(two, 232)","(study, 226)","(one, 222)","(liver, 221)"
6,C07,"(patients, 116)","(treatment, 55)","(periodontal, 44)","(disease, 37)","(gland, 36)","(primary, 36)","(dental, 35)","(cells, 34)","(patient, 34)","(parotid, 34)"
7,C08,"(patients, 1000)","(lung, 424)","(pulmonary, 399)","(less, 268)","(disease, 250)","(+/-, 245)","(respiratory, 232)","(group, 220)","(treatment, 212)","(patient, 177)"
8,C09,"(patients, 201)","(disease, 56)","(treatment, 55)","(ear, 55)","(nasal, 54)","(study, 50)","(cases, 46)","(hearing, 46)","(group, 45)","(one, 42)"
9,C10,"(patients, 1298)","(group, 253)","(pain, 248)","(may, 245)","(less, 244)","(patient, 238)","(disease, 230)","(study, 209)","(clinical, 202)","(treatment, 194)"


#### Challenges:
#there is words which common at most of the categories (such as ),that can effect the accurcy of the model.
#finding the right parameters and understands there meaning in order to impprove the algorithem
# 

<h1> 2. Document classification: <h1>

feature extraction methods: TF-IDF, Bag Of Words
machine learning models: SVM, Naïve Bayes

In [19]:
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron

In [20]:
# save best
best_accuracy = 0.0
best_fe = ''
best_classifier = ''
best_corpus_type = ''

In [21]:
train=docInfoDF.loc[docInfoDF['group']=='training']
trainSetText=train['cleanText'].tolist()
trainSetClass=train['class'].tolist()
test=pd.dataFrame(testInfo, columns=['fileName','cleanText','class','group'])
testSetText=train['cleanText'].tolist()
testSetClass=train['class'].tolist()

In [22]:
def classify(extractM,machineLearningM ): 
            pipeline = Pipeline([('vect', extractM), ('clf', machineLearningM)])
            parameters = {'vect__max_df': np.arange(0.05, 0.55, 0.05),'clf__alpha': np.arange(0.01, 0.11, 0.01)}
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(trainSetText, trainSetClass)
            prediction = gs_clf.predict(testSetText)
            accuracy = metrics.accuracy_score(testSetClass, prediction)
            print(accuracy)
            return accuracy

In [23]:
results=[]
#TF-IDF & SVM
print('TF-IDF AND SVM')
accuracy = classify(TfidfVectorizer(),SGDClassifier() )
results.append(['TF-IDF' , 'SVM', accuracy])


TF-IDF AND SVM


KeyboardInterrupt: 

In [ ]:
#TF-IDF & Naïve Bayes
print('TF-IDF AND Naïve Bayes')
accuracy = classify(TfidfVectorizer(),MultinomialNB() )
results.append(['TF-IDF' , 'Naïve Bayes', accuracy])

In [ ]:
#Bag of Words & SVM
print('Bag of Words AND SVM')
accuracy = classify(CountVectorizer(),SGDClassifier() )
results.append(['Bag of Words' , 'SVM', accuracy])

In [ ]:
#Bag of Words & Naïve Bayes
print('Bag of Words AND Naïve Bayes')
accuracy = classify(CountVectorizer(),MultinomialNB() )
results.append(['Bag of Words' , 'Naïve Bayes', accuracy])


In [ ]:
pd.DataFrame(results,columns=['extraction method','machine learning method', 'accuracy'])